In [1]:
import sys

sys.path.append('/usr/local/spark/python/lib/py4j-0.10.3-src.zip')
sys.path.remove('/usr/local/spark/python/lib/py4j-0.9-src.zip')

In [2]:
import pyspark
conf = pyspark.SparkConf()

conf.setMaster("mesos://dashboard.me/service/spark:5050")
conf.setAppName("My Test App")
# conf.set("spark.executor.uri", "http://d3kbcqa49mib13.cloudfront.net/spark-2.0.2-bin-hadoop2.6.tgz")
# conf.set("spark.mesos.executor.home", "/opt/spark/dist")

In [3]:
# create the context
sc = pyspark.SparkContext(conf=conf)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.3-src.zip/py4j/java_gateway.py", line 883, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.3-src.zip/py4j/java_gateway.py", line 1040, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:35062)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.3-src.zip/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
  File "/opt/conda/envs/python2/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:35062)
Traceback (most rec

Py4JError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext

In [ ]:
rd = sc.parallelize(range(1000))
rd.count()

##### Example

A table contains

|UID|ACCESS_TIME|
| ------| ------ | ------ |
|A|1::1234|
|B|1::2345|
|C|1::2367|
|A|1::3456|
|B|1::4567|

Find minimum interval for each user.

In [4]:
table = sc.parallelize([('A', 123), ('B', 456), ('C', 678), ('A', 789), ('B', 889)])

In [5]:
table.sortBy(lambda x: x[1]).sortByKey(ascending=True).collect()

[('A', 789), ('A', 123), ('B', 456), ('B', 889), ('C', 678)]

In [22]:
table.co

In [23]:
groups.foreachPartition?

##### Using Aggregate Find Mean

- aggregate(zeroValue: U)(seqOp: (U, T) ⇒ U, combOp: (U, U) ⇒ U): U

- The first function (seqOp) can return a different result type, U, than
the type of this RDD type T. Thus, we need one operation for merging a T into
an U and one operation for merging two U.

In [6]:
var = sc.parallelize(xrange(0, 11))

In [9]:
(sum, counter) = var.aggregate((0, 0), seqOp=lambda x, y: (x[0] + y, x[1] + 1), combOp=lambda x, y: (x[0] + y[0], x[1] + y[1]))
sum/counter

5

##### Understanding Closures

``` python
counter = 0
rdd = sc.parallelize(data)

# Wrong: Don't do this!!
def increment_counter(x):
    global counter
    counter += x
rdd.foreach(increment_counter)

print("Counter value: ", counter)
```

To execute jobs,
1. Computes the task’s closure
2. Breaks up the processing of RDD operations into tasks, each of process which is executed by an executor.
3. This closure is serialized and sent to each executor.

There is still a counter in the memory of the driver node but this is no longer visible to the executors! The executors only see the copy from the serialized closure.

Then the [accumulator](http://spark.apache.org/docs/latest/programming-guide.html#accumulators)

##### Spark Cluster Overview

In [3]:
from IPython.display import Image
Image(url='http://spark.apache.org/docs/latest/img/cluster-overview.png')

##### Note

- if read file error, try try sudo and abs path. 

- rdd.take(n) (return list) shows content.

##### RDD api has two types:

- [Transformations](http://spark.apache.org/docs/latest/programming-guide.html#transformations)

- [Action](http://spark.apache.org/docs/latest/programming-guide.html#actions)

##### Shared Variables

[Reference](http://spark.apache.org/docs/latest/programming-guide.html#broadcast-variables)

Normally, when a function passed to a Spark operation (such as map or reduce) **is executed on a remote cluster node**, it works on separate copies of all the variables used in the function. **These variables are copied to each machine, and no updates** to the variables on the remote machine are propagated back to the driver program.

- Broadcast Variables : A read-only variable cached on each machine.

- Accumulators : Variables that are only “added” to through an associative operation.

##### Estimating Pi Example

```python
def sample(p):
    x, y = random(), random()
    return 1 if x*x + y*y < 1 else 0

count = spark.parallelize(xrange(0, NUM_SAMPLES)).map(sample) \
             .reduce(lambda a, b: a + b)
print "Pi is roughly %f" % (4.0 * count / NUM_SAMPLES)
```

##### Logistic Regression Example

```python
points = spark.textFile(...).map(parsePoint).cache()
w = numpy.random.ranf(size = D) # current separating plane
for i in range(ITERATIONS):
    gradient = points.map(
        lambda p: (1 / (1 + exp(-p.y*(w.dot(p.x)))) - 1) * p.y * p.x
    ).reduce(lambda a, b: a + b)
    w -= gradient
print "Final separating plane: %s" % w
```

##### Reference

[RDD](http://spark.apache.org/docs/latest/programming-guide.html#using-the-shell)

[Very Useful RDD Operation](https://spark.apache.org/docs/0.7.2/api/core/spark/RDD.html)

[Packages](https://spark-packages.org/)

[Contributing](http://spark.apache.org/contributing.html)